# n-Player FPSB auctions

## Imports

In [15]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
from timeit import default_timer as timer

In [16]:
import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

In [17]:
from bnelearn.strategy import NeuralNetStrategy, TruthfulStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import AuctionEnvironment

In [18]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

# set up matplotlib
is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display
#
#plt.ion()

In [5]:
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

# Use specific cuda gpu if desired (i.e. for running multiple experiments in parallel)
specific_gpu = None
if cuda and specific_gpu:
    torch.cuda.set_device(specific_gpu)


print(device)

cuda


## Settings

The following cell fully defines an experiment

In [9]:
run_dir = 'fpsb_3p_sym_uniform'
run_name = 'test'
logdir = os.path.join(root_path, 'notebooks', run_dir , run_name)

## Experiment setup
n_players = 3
n_items = 1
# valuation distribution
u_lo =0
u_hi =10

def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo, u_hi, strategy, batch_size = batch_size, n_players=5)

#def strat_to_bidder(strategy, batch_size):
#    return Bidder.normal(10.0, 5.0, strategy, batch_size = batch_size, n_players=1)

## Environment settings
#training batch size
batch_size = 2**12
input_length = 1

# strategy model architecture
size_hidden_layer = 10

# optimization params
epoch = 1000
learning_rate = 1e-1
lr_decay = True
lr_decay_every = 1000
lr_decay_factor = 0.7
baseline = True
momentum = 0.5

sigma = .02 #ES noise parameter
n_perturbations = 64

# plot and log training options
plot_epoch = 100
plot_points = min(100, batch_size)

plot_xmin = u_lo
plot_xmax = u_hi
plot_ymin = 0
plot_ymax = 10

#sample_points = torch.from_numpy(np.linspace(u_lo, u_hi, u_hi+1)).float().view(-1, n_items).cuda()

# tensorboard writer settings

## Setting up the Environment

In [10]:
# for evaluation
def optimal_bid(valuation):
    return valuation * (n_players - 1) / n_players

def log_hyperparams(writer, e):
    writer.add_scalar('hyperparams/batch_size', batch_size, e)
    writer.add_scalar('hyperparams/size_hidden_layer', size_hidden_layer, 0)
    writer.add_scalar('hyperparams/learning_rate', learning_rate, e)
    writer.add_scalar('hyperparams/momentum', momentum, e)
    writer.add_scalar('hyperparams/sigma', sigma, e)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations, e)

v_opt = np.linspace(plot_xmin, plot_xmax, 100)
b_opt = optimal_bid(v_opt)
    
def plot_bid_function(fig, v,b, writer=None, e=None, plot_points=100):
    
    # subsample points and plot
    v = v.detach().cpu().numpy()[:plot_points]
    b= b.detach().cpu().numpy()[:plot_points]
    
    fig = plt.gcf()
    plt.cla()
    plt.xlim(plot_xmin, plot_xmax)
    plt.ylim(plot_ymin, plot_ymax)
    plt.plot(v,b, 'o', v_opt, b_opt, 'r--')
    #if is_ipython:
        #display.clear_output(wait=True)
    display.display(plt.gcf())
    if writer:
        writer.add_figure('eval/bid_function', fig, e)  

In [11]:
model = NeuralNetStrategy(input_length,
                          size_hidden_layer = size_hidden_layer,
                          requires_grad=False
                         ).to(device)

bidder1 = strat_to_bidder(model, batch_size)
bidder2 = strat_to_bidder(model, batch_size)
bidder3 = strat_to_bidder(model, batch_size)

In [12]:
mechanism = FirstPriceSealedBidAuction(cuda = True)
env = AuctionEnvironment(mechanism,
                  agents = [bidder1, bidder2, bidder3], #dynamically built
                  max_env_size = 3, #
                  batch_size = batch_size,
                  n_players =n_players,
                  strategy_to_bidder_closure = strat_to_bidder
                 )
optimizer = ES(model=model, environment = env,
               lr = learning_rate, momentum=momentum,
               sigma=sigma, n_perturbations=n_perturbations,
               baseline=baseline)

## Training

In [13]:
with SummaryWriter(logdir, flush_secs=120) as writer:
    torch.cuda.empty_cache()
    log_hyperparams(writer, 0)
    fig = plt.figure()
    for e in range(epoch+1):
        # lr decay?
        if lr_decay and e % lr_decay_every == 0 and e > 0:
            learning_rate = learning_rate * lr_decay_factor
            log_hyperparams(writer, e)
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate

        # always: do optimizer step
        utility = -optimizer.step()
        writer.add_scalar('eval/utility', utility, e)

        if e % plot_epoch == 0:
            start_time = timer()
            # log statistics and plot current function output
            bidder = strat_to_bidder(model, batch_size)
            bidder.draw_valuations_()
            v = bidder.valuations
            b = bidder.get_action()
            share = b.mean()/optimal_bid(v).mean()
            diff = (b-optimal_bid(v)).mean()
            writer.add_scalar('eval/share', share, e)
            writer.add_scalar('eval/diff', diff, e) 
            writer.add_graph(model, bidder.valuations)            
                    
            print("Epoch {}: \ttotal share: {:.3f}, diff: {:.3f}, \tutility: {:.3f}".format(e, share, diff, utility))
            plot_bid_function(fig, v,b,writer,e)
            
            elapsed = timer() - start_time
            print("Logging checkpoint took {:.2f}s.".format(elapsed))
            

RuntimeError: shape '[4096, 3, 1]' is invalid for input of size 8192

<Figure size 432x288 with 0 Axes>

In [19]:
s = [1,2,3]

In [20]:
e = None

In [22]:
[a for a in s if a not in set(e)]

TypeError: 'NoneType' object is not iterable

In [25]:
from collections import deque
x = deque()

In [32]:
if not x:
    print('hi')

hi


In [34]:
set([5])

{5}

In [36]:
x = torch.zeros(3, 2, 4)

In [38]:
x[:, 1, :] = torch.ones(3,4)

In [39]:
x

tensor([[[0., 0., 0., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [1., 1., 1., 1.]]])

In [49]:
def genf():
    for i in range(10):
        yield (i,i)

In [50]:
gen = genf()

In [42]:
next(gen)

(0, 0)

In [51]:
for (a,b) in gen:
    print(a,b)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9


In [54]:
x.shape[2]

4

In [57]:
dynamic = object()

In [60]:
x = 'hello'

In [61]:
x is 'hello'

True